In [10]:
# %reset -f
# %load_ext autoreload
# %autoreload 2

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import torch
from IPython.display import HTML
from pybounds import Simulator

In [11]:
from double_pendulum import DoublePendulumParameters, DoublePendulumModel, DoublePendulumDrawer, ArrowDrawer

# Set model parameters

In [12]:
dt = 0.05  #  time-step [s]

In [13]:
params = DoublePendulumParameters(
    g=0.0,  # gravity
    k_1=0.0, k_2=1.0,  # torsional stiffness
    c_1=1.0, c_2=1.0,  # torsional damping
    L_1=1.0, L_2=1.0,  # length
    m_1=1.0, m_2=1.0,  # mass
    c_w_1=2.0, c_w_2=2.0,  # wind damping
    J_1=None, J_2=None,  # inertia automatically computed based on cylinder model
)
model = DoublePendulumModel(parameters=params)
model.parameters.__dict__

{'g': 0.0,
 'L_1': 1.0,
 'L_2': 1.0,
 'k_1': 0.0,
 'k_2': 1.0,
 'c_1': 1.0,
 'c_2': 1.0,
 'm_1': 1.0,
 'm_2': 1.0,
 'c_w_1': 2.0,
 'c_w_2': 2.0,
 'J_1': 0.08333333333333333,
 'J_2': 0.08333333333333333}

# Construct simulator object

In [14]:
simulator = Simulator(model.f, model.h,
                      dt=dt,
                      state_names=model.state_names,
                      input_names=model.input_names,
                      measurement_names=model.measurement_names)

# Set inputs

In [15]:
T = 5.0  # total time [s]
tsim = np.arange(start=0.0, stop=T + 1e-3, step=dt)  # time vector [s]

In [ ]:
NA = np.zeros_like(tsim)
setpoint = {'x': 0.0 * np.ones_like(tsim),
            'y': 0.0 * np.ones_like(tsim),
            'theta_1': (np.pi/4) * np.ones_like(tsim),
            'theta_2': (np.pi/4) * np.ones_like(tsim),
            'w': 1.0 * np.ones_like(tsim),
            'zeta': (-np.pi/2) * np.ones_like(tsim),
            'x_dot': NA,
            'y_dot': NA,
            'theta_dot_1': NA,
            'theta_dot_2': NA,
            }
# Inputs
u = {'x_ddot': -0.05 + 0.2*np.cos(2*np.pi*(1/T) * tsim),  # x-acceleration of base
     'y_ddot': -0.05 + 0.2*np.sin(2*np.pi*(1/T) * tsim),  # y-acceleration of base
     'tau_1': 0.0*np.ones_like(tsim),  # torque on 1st segment
     'tau_2': 0.0*np.ones_like(tsim), # torque on 2nd segment
     }
# Update the simulator set-point
simulator.update_dict(setpoint, name='setpoint')

# Set up model predictive control

In [17]:
# Define cost function
cost = ((simulator.model.x['x'] - simulator.model.tvp['x_set']) ** 2 +
        (simulator.model.x['y'] - simulator.model.tvp['y_set']) ** 2 +
        (simulator.model.x['theta_1'] - simulator.model.tvp['theta_1_set']) ** 2
        )

# Set cost function
simulator.mpc.set_objective(mterm=cost, lterm=cost)

In [18]:
# Set input penalty: make this small for accurate state following
simulator.mpc.set_rterm(x_ddot=1e-6, y_ddot=1e-6, tau_1=1e-6, tau_2=1e12)

# Run simulation in closed-loop

In [19]:
t_sim, x_sim, u_sim, y_sim = simulator.simulate(mpc=True,  return_full_output=True)
sim_data = pd.DataFrame(y_sim)
sim_data.insert(0, 'time', t_sim)
sim_data

,time,theta_1,theta_2,theta_dot_1,theta_dot_2,x,y,x_dot,y_dot,w,zeta,x_ddot,y_ddot,tau_1,tau_2,theta_2_1
0,0.00,0.785398,0.785398,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,-1.570796,1.281093e-02,-1.277390e-02,-0.169956,5.700618e-19,0.000000
1,0.05,0.785494,0.782775,-0.000552,-0.094811,1.601366e-05,-1.596737e-05,6.405464e-04,-6.386949e-04,1.0,-1.570796,-1.180321e-02,1.168533e-02,0.042164,6.165016e-19,-0.002719
2,0.10,0.785476,0.776090,-0.003438,-0.165025,3.328697e-05,-3.329546e-05,5.038606e-05,-5.442861e-05,1.0,-1.570796,-1.060391e-02,1.065378e-02,0.315615,5.144847e-19,-0.009386
3,0.15,0.785414,0.766129,-0.002134,-0.226417,2.255138e-05,-2.269967e-05,-4.798096e-04,4.782602e-04,1.0,-1.570796,3.252335e-03,-3.164888e-03,0.525147,4.472785e-19,-0.019285
4,0.20,0.785397,0.753424,-0.001099,-0.275967,2.626321e-06,-2.742767e-06,-3.171928e-04,3.200158e-04,1.0,-1.570796,6.222993e-03,-6.237549e-03,0.681833,4.174758e-19,-0.031973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,4.80,0.785398,-0.173216,0.000001,-0.022587,-4.630290e-08,5.701493e-08,4.092852e-08,-9.032448e-09,1.0,-1.570796,-2.433455e-07,1.209088e-07,1.694782,-9.584900e-20,-0.958614
97,4.85,0.785398,-0.174318,0.000001,-0.021495,-4.456065e-08,5.671444e-08,2.876124e-08,-2.987007e-09,1.0,-1.570796,-2.699141e-07,2.241960e-07,1.694471,-1.067408e-19,-0.959716
98,4.90,0.785398,-0.175366,0.000001,-0.020453,-4.345998e-08,5.684534e-08,1.526553e-08,8.222791e-09,1.0,-1.570796,-1.155483e-07,6.913078e-08,1.694173,-1.171828e-19,-0.960764
99,4.95,0.785398,-0.176364,0.000001,-0.019460,-4.284114e-08,5.734289e-08,9.488120e-09,1.167933e-08,1.0,-1.570796,5.812302e-08,-1.436667e-07,1.693887,-1.271883e-19,-0.961762


# Animate

In [20]:
plt.rcParams['animation.embed_limit'] = 100

In [21]:
drawer = DoublePendulumDrawer(
    sim_data['x'].values,
    sim_data['y'].values,
    sim_data['theta_1'].values,
    sim_data['theta_2'].values,
    L1=model.parameters.L_1,
    L2=model.parameters.L_2,
    seg1_kwargs=dict(color='blue', linewidth=4.0, markersize=7),
    seg2_kwargs=dict(color='red', linewidth=4.0, markersize=7),
    trail1=True, trail2=True, trail_base=True
)

wind_arrow = ArrowDrawer(origin=(drawer.get_axis_bounds()[2] + 2.5, 1.25), magnitude=0.5, angle=0, color='green', width=0.05)

In [22]:
# Make figure
fig, ax = plt.subplots(nrows=1, ncols=1)

def update(frame):
    ax.clear()
    drawer.draw(ax, frame=frame)
    wind_arrow.draw(ax, angle=sim_data['zeta'].values[frame])
    ax.set_aspect('equal')
    ax.axis(drawer.get_axis_bounds())
    ax.set_xlim(right=ax.axes.get_xlim()[-1] + 1.0)

# Make animation
animation = mpl.animation.FuncAnimation(
    fig, update,
    frames=sim_data.shape[0],
    blit=False,
    interval=int(1000 * dt))

# update(0)
plt.close(fig)

In [23]:
# Display it
HTML(animation.to_jshtml())

In [24]:
# # Save it
# from matplotlib.animation import PillowWriter
# save_path = os.path.join(os.path.pardir, 'animation', 'example_model predictive_control_wind.gif')
# animation.save(save_path, PillowWriter(fps=int(1/dt)))